<a href="https://colab.research.google.com/github/tg-bomze/collection-of-notebooks/blob/master/VCR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<b><font color="black" size="+4">RefColor v.1.0</font></b>

![visitors](https://visitor-badge.glitch.me/badge?page_id=mlart.RefColor&left_color=black&right_color=orange)

<b><font color="black" size="+2">Based on:</font></b>

**GitHub repository**: [Deep-Exemplar-based-Video-Colorization](https://github.com/zhangmozhe/Deep-Exemplar-based-Video-Colorization), [BLIP](https://github.com/salesforce/BLIP)

Creator: **[zhangmozhe](https://github.com/zhangmozhe), [salesforce](https://github.com/salesforce)**

<b><font color="black" size="+2">Colab created by:</font></b>

GitHub: [@tg-bomze](https://github.com/tg-bomze),
Telegram: [@MLArt](https://t.me/MLArt),
Twitter: [@tg_bomze](https://twitter.com/tg_bomze).

```
(ENG) To get started, click on the button (where the red arrow indicates). After clicking, wait until the execution is complete.
```
```
(RUS) Чтобы начать, поочередно нажимайте на кнопки (куда указывают красные стрелки), дожидаясь завершения выполнения каждого блока.
```

In [ ]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Install all necessary libraries</font></b>
#@markdown <b><font color="black" size="+1"> (RUS) Установить все необходимые библиотеки</font></b>

import os
import cv2
import glob
from tqdm import tqdm
import IPython
from IPython.display import clear_output
from google.colab import output
output.disable_custom_widget_manager()
%cd /content
!git clone https://github.com/zhangmozhe/Deep-Exemplar-based-Video-Colorization.git DEVC
%cd /content/DEVC
!pip install --upgrade scenedetect[opencv]
!pip install -r requirements.txt
!wget https://facevc.blob.core.windows.net/zhanbo/old_photo/colorization_checkpoint.zip
!unzip colorization_checkpoint.zip

def color_restore(orig, color):
    orig_yuv = cv2.cvtColor(orig, cv2.COLOR_BGR2YUV)
    color_yuv = cv2.cvtColor(color, cv2.COLOR_BGR2YUV)
    orig_yuv[:, :, 1:3] = color_yuv[:, :, 1:3]
    return cv2.cvtColor(orig_yuv, cv2.COLOR_YUV2BGR)

try: 
  !pip3 install googletrans==3.1.0a0
  from googletrans import Translator, constants
  from pprint import pprint
  translator = Translator()
except: pass

!pip install bing-image-downloader
from bing_image_downloader import downloader

%cd /content
!pip install transformers==4.15.0 timm==0.4.12 fairscale==0.4.4 scipy==1.7.3
!git clone https://github.com/salesforce/BLIP
%cd /content/BLIP
import PIL
from PIL import Image
import torch
from torchvision import transforms
from torchvision.transforms.functional import InterpolationMode
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
from models.blip import blip_decoder
model_blip = blip_decoder(pretrained='https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_base_capfilt_large.pth', image_size=384, vit='base')
model_blip.eval()
model_blip = model_blip.to(device)
%cd /content
transform = transforms.Compose([
        transforms.Resize((384,384),interpolation=InterpolationMode.BICUBIC),
        transforms.ToTensor(),
        transforms.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))
        ]) 

if os.path.isfile("/content/DEVC/colorization_checkpoint.zip"):
  print('Done!')
  clear_output()

In [ ]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+2"> Edit /content/DEVC/test.py</font></b>
#@markdown <b><font color="black" size="+1"> (RUS) Исправить файл /content/DEVC/test.py</font></b>
%%writefile /content/DEVC/test.py
from __future__ import print_function

import argparse
import glob
import os
import time

import cv2
import numpy as np
import torch
import torch.backends.cudnn as cudnn
import torchvision.transforms as transform_lib
from PIL import Image
from tqdm import tqdm

import lib.TestTransforms as transforms
from models.ColorVidNet import ColorVidNet
from models.FrameColor import frame_colorization
from models.NonlocalNet import VGG19_pytorch, WarpNet
from utils.util import (batch_lab2rgb_transpose_mc, folder2vid, mkdir_if_not,
                        save_frames, tensor_lab2rgb, uncenter_l)
from utils.util_distortion import CenterPad, Normalize, RGB2Lab, ToTensor

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
torch.cuda.set_device(0)


def colorize_video(opt, input_path, reference_file, output_path, nonlocal_net, colornet, vggnet):
    # parameters for wls filter
    wls_filter_on = True
    lambda_value = 500
    sigma_color = 4

    # processing folders
    mkdir_if_not(output_path)
    files = glob.glob(output_path + "*")
    print("processing the folder:", input_path)
    path, dirs, filenames = os.walk(input_path).__next__()
    file_count = len(filenames)
    filenames.sort(key=lambda f: int("".join(filter(str.isdigit, f) or -1)))

    # NOTE: resize frames to 216*384
    transform = transforms.Compose(
        [CenterPad(opt.image_size), transform_lib.CenterCrop(opt.image_size), RGB2Lab(), ToTensor(), Normalize()]
    )

    # if frame propagation: use the first frame as reference
    # otherwise, use the specified reference image
    ref_name = input_path + filenames[0] if opt.frame_propagate else reference_file
    print("reference name:", ref_name)
    frame_ref = Image.open(ref_name)

    total_time = 0
    I_last_lab_predict = None

    IB_lab_large = transform(frame_ref).unsqueeze(0).cuda()
    IB_lab = torch.nn.functional.interpolate(IB_lab_large, scale_factor=0.5, mode="bilinear")
    IB_l = IB_lab[:, 0:1, :, :]
    IB_ab = IB_lab[:, 1:3, :, :]
    with torch.no_grad():
      I_reference_lab = IB_lab
      I_reference_l = I_reference_lab[:, 0:1, :, :]
      I_reference_ab = I_reference_lab[:, 1:3, :, :]
      I_reference_rgb = tensor_lab2rgb(torch.cat((uncenter_l(I_reference_l), I_reference_ab), dim=1))
      features_B = vggnet(I_reference_rgb, ["r12", "r22", "r32", "r42", "r52"], preprocess=True)

    for index, frame_name in enumerate(tqdm(filenames)):
        frame1 = Image.open(os.path.join(input_path, frame_name))
        IA_lab_large = transform(frame1).unsqueeze(0).cuda()
        IA_lab = torch.nn.functional.interpolate(IA_lab_large, scale_factor=0.5, mode="bilinear")

        IA_l = IA_lab[:, 0:1, :, :]
        IA_ab = IA_lab[:, 1:3, :, :]
        
        if I_last_lab_predict is None:
            if opt.frame_propagate:
                I_last_lab_predict = IB_lab
            else:
                I_last_lab_predict = torch.zeros_like(IA_lab).cuda()

        # start the frame colorization
        with torch.no_grad():
            I_current_lab = IA_lab
            I_current_ab_predict, I_current_nonlocal_lab_predict, features_current_gray = frame_colorization(
                I_current_lab,
                I_reference_lab,
                I_last_lab_predict,
                features_B,
                vggnet,
                nonlocal_net,
                colornet,
                feature_noise=0,
                temperature=1e-10,
            )
            I_last_lab_predict = torch.cat((IA_l, I_current_ab_predict), dim=1)

        # upsampling
        curr_bs_l = IA_lab_large[:, 0:1, :, :]
        curr_predict = (
            torch.nn.functional.interpolate(I_current_ab_predict.data.cpu(), scale_factor=2, mode="bilinear") * 1.25
        )

        # filtering
        if wls_filter_on:
            guide_image = uncenter_l(curr_bs_l) * 255 / 100
            wls_filter = cv2.ximgproc.createFastGlobalSmootherFilter(
                guide_image[0, 0, :, :].cpu().numpy().astype(np.uint8), lambda_value, sigma_color
            )
            curr_predict_a = wls_filter.filter(curr_predict[0, 0, :, :].cpu().numpy())
            curr_predict_b = wls_filter.filter(curr_predict[0, 1, :, :].cpu().numpy())
            curr_predict_a = torch.from_numpy(curr_predict_a).unsqueeze(0).unsqueeze(0)
            curr_predict_b = torch.from_numpy(curr_predict_b).unsqueeze(0).unsqueeze(0)
            curr_predict_filter = torch.cat((curr_predict_a, curr_predict_b), dim=1)
            IA_predict_rgb = batch_lab2rgb_transpose_mc(curr_bs_l[:32], curr_predict_filter[:32, ...])
        else:
            IA_predict_rgb = batch_lab2rgb_transpose_mc(curr_bs_l[:32], curr_predict[:32, ...])

        # save the frames
        save_frames(IA_predict_rgb, output_path, index)

    # output video
    video_name = "video.avi"
    folder2vid(image_folder=output_path, output_dir=output_path, filename=video_name)
    print()

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--frame_propagate", default=False, type=bool, help="propagation mode, , please check the paper"
    )
    parser.add_argument("--image_size", type=int, default=[216 * 2, 384 * 2], help="the image size, eg. [216,384]")
    parser.add_argument("--cuda", action="store_false")
    parser.add_argument("--gpu_ids", type=str, default="0", help="separate by comma")
    parser.add_argument("--clip_path", type=str, default="./sample_videos/clips/v32", help="path of input clips")
    parser.add_argument("--ref_path", type=str, default="./sample_videos/ref/v32", help="path of refernce images")
    parser.add_argument("--output_path", type=str, default="./sample_videos/output", help="path of output clips")
    opt = parser.parse_args()
    opt.gpu_ids = [int(x) for x in opt.gpu_ids.split(",")]
    cudnn.benchmark = True
    print("running on GPU", opt.gpu_ids)

    clip_name = opt.clip_path.split("/")[-1]
    refs = os.listdir(opt.ref_path)
    refs.sort()

    nonlocal_net = WarpNet(1)
    colornet = ColorVidNet(7)
    vggnet = VGG19_pytorch()
    vggnet.load_state_dict(torch.load("data/vgg19_conv.pth"))
    for param in vggnet.parameters():
        param.requires_grad = False

    nonlocal_test_path = os.path.join("checkpoints/", "video_moredata_l1/nonlocal_net_iter_76000.pth")
    color_test_path = os.path.join("checkpoints/", "video_moredata_l1/colornet_iter_76000.pth")
    print("succesfully load nonlocal model: ", nonlocal_test_path)
    print("succesfully load color model: ", color_test_path)
    nonlocal_net.load_state_dict(torch.load(nonlocal_test_path))
    colornet.load_state_dict(torch.load(color_test_path))

    nonlocal_net.eval()
    colornet.eval()
    vggnet.eval()
    nonlocal_net.cuda()
    colornet.cuda()
    vggnet.cuda()

    for ref_name in refs:
        try:
            colorize_video(
                opt,
                opt.clip_path,
                os.path.join(opt.ref_path, ref_name),
                os.path.join(opt.output_path, clip_name + "_" + ref_name.split(".")[0]),
                nonlocal_net,
                colornet,
                vggnet,
            )
        except Exception as error:
            print("error when colorizing the video " + ref_name)
            print(error)

In [ ]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Prepare video</font></b>
#@markdown <b><font color="black" size="+1"> (RUS) Подготовка видео</font></b>
path_to_video = '' #@param {type:"string"}

!rm -rf /content/i.mp4 /content/audio.aac
try:
  !ffmpeg -y -i "$path_to_video" -vn -acodec copy /content/audio.aac -hide_banner -loglevel error
except: pass
fps_of_video = round(cv2.VideoCapture(path_to_video).get(cv2.CAP_PROP_FPS))
!ffmpeg -i "$path_to_video" -vf format=gray /content/i.mp4 -hide_banner -loglevel error
%cd /content
!rm -rf /content/scenes $path_to_video
!mkdir /content/scenes
%cd /content/scenes
if os.path.isfile("/content/i.mp4"):
  print('\t Start splitting video into scenes!')
  clear_output()
!scenedetect -i /content/i.mp4 split-video detect-content -t 10
if len(os.listdir('/content/scenes')) >= 1:
  print('Done!')
  clear_output()
#!scenedetect -i /content/i.mp4 --stats /content/stats.csv detect-content

In [ ]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Colorize</font></b>
#@markdown <b><font color="black" size="+1"> (RUS) Колоризовать</font></b>
control = False #@param {type:"boolean"}
!rm -rf /content/DEVC/sample_videos/* /content/temp/*
!mkdir /content/DEVC/sample_videos/result /content/DEVC/sample_videos/output
scenes_name = os.listdir('/content/scenes')
scenes_name.sort()
%cd /content/DEVC
count = 1
for i in tqdm(scenes_name):
  only_name = i.split('.')[0]
  !mkdir -p "/content/DEVC/sample_videos/clips/$only_name"
  !mkdir -p "/content/DEVC/sample_videos/ref/$only_name"
  !ffmpeg -y -i "/content/scenes/$i" "/content/DEVC/sample_videos/clips/$only_name/%7d.png" -hide_banner -loglevel error

  raw_image = Image.open(f'/content/DEVC/sample_videos/clips/{only_name}/0000001.png').convert('RGB')
  image = transform(raw_image).unsqueeze(0).to(device)
  with torch.no_grad():
    caption = model_blip.generate(image, sample=False, num_beams=3, max_length=20, min_length=5) 
    true_caption = caption[0].replace('in black and white', '')
    true_caption += ' color photo'
  if control:
    text = input(f'\n /content/DEVC/sample_videos/clips/{only_name}/0000001.png is "{true_caption}":')
    if text != '':
      try:
        translator = Translator()
        translation = translator.translate(text)
        true_caption = translation.text
      except: true_caption = text 
  downloaded_img_path = '/content/some'
  while true_caption != '':
    !rm -rf /content/temp/* $downloaded_img_path
    downloader.download(true_caption, limit=1,  output_dir=f'/content/temp', adult_filter_off=True, force_replace=False, timeout=60, verbose=False)
    downloaded_img_path = glob.glob(f'/content/temp/{true_caption}/*.*')[0]
    if control:
      text = input(f'\n {downloaded_img_path} - is normal image? ')
      try:
        translator = Translator()
        translation = translator.translate(text)
        true_caption = translation.text
      except: true_caption = text 
    else: true_caption = ''
  !cp "$downloaded_img_path" "/content/DEVC/sample_videos/ref/$only_name/"

  !python test.py --clip_path "./sample_videos/clips/$only_name" \
               --ref_path "./sample_videos/ref/$only_name" \
               --output_path "./sample_videos/output" \
               --cuda --gpu_ids 0
  for_del = f'/content/DEVC/sample_videos/output/{only_name}_Image_1/video.avi'
  !rm -rf $for_del
  
  frame_list = glob.glob(f'/content/DEVC/sample_videos/output/{only_name}_Image_1/*.*')
  frame_list.sort()
  for frame in tqdm(frame_list):
    color = cv2.imread(frame)
    frame_name = f'/content/DEVC/sample_videos/clips/{only_name}/' + str(int(os.path.basename(frame).split('.')[0])+1).zfill(7) + '.png'
    orig = cv2.imread(frame_name)
    color = cv2.resize(color, (orig.shape[1], orig.shape[0]))
    result = color_restore(orig, color)
    result_name = str(len(glob.glob('/content/DEVC/sample_videos/result/*.*'))).zfill(7)
    cv2.imwrite(f'/content/DEVC/sample_videos/result/{result_name}.jpg', result)

  if len(os.listdir('/content/DEVC/sample_videos/output')) == count:
    count += 1
    clear_output()

In [ ]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Get result</font></b>
#@markdown <b><font color="black" size="+1"> (RUS) Получить результат</font></b>
if os.path.isfile("/content/audio.aac"):
  !ffmpeg -y -r $fps_of_video -i /content/DEVC/sample_videos/result/%07d.jpg -c:v libx264 -vf fps=$fps_of_video -pix_fmt yuv420p /content/temp_result.mp4 -hide_banner -loglevel error
  !ffmpeg -y -i /content/temp_result.mp4 -i /content/audio.aac /content/result.mp4 -hide_banner -loglevel error
  !rm -rf /content/temp_result.mp4
else: 
  !ffmpeg -y -r $fps_of_video -i /content/DEVC/sample_videos/result/%07d.jpg -c:v libx264 -vf fps=$fps_of_video -pix_fmt yuv420p /content/result.mp4 -hide_banner -loglevel error

print('Result here: /content/result.mp4')